In [1]:
from gpt import *

In [2]:
gin.parse_config_file("config/gpt2-small.gin")
config = GPTConfig()
config

GPTConfig(block_size=1024, vocab_size=50257, n_layer=12, n_head=12, n_embd=768, batch_size=16, learning_rate=0.0003, seed=42, epochs=1)

In [ ]:
model = GPT.from_pretrained()